In [2]:
import jet
import numpy as np

In [3]:
# Define a quantum circuit with 2 wires
# Initialized with 2 qubits in zero state
# This contraction happens with the defined scalars (0-state initialized qubits)
qc = jet.Circuit(2)
qc.append_gate(jet.Hadamard(), [0])
qc.append_gate(jet.CX(), [0,1])
qc.append_gate(jet.PauliZ(), [0])
qc.append_gate(jet.CX(), [0,1])
qc.append_gate(jet.Hadamard(), [0])
qc.append_state(jet.state.Qubit(np.array([1+0j, 0])), [0]) 
qc.append_state(jet.state.Qubit(np.array([1+0j, 0])), [1])
tn = qc.tensor_network()

In [4]:
tn.contract()

Size = 1
Indices = {}
Data = {(0,0)}

In [29]:
# Define all gates for test circuit
angle = (np.pi/4)*1j
angle_p = -(np.pi/4) * 1j
T = jet.Tensor(["8","9"], [2,2], [1,0,0,np.exp(angle)])
T_p = jet.Tensor(["0","1"], [2,2],[1,0,0,np.exp(angle_p)])

# Defining CNOT gate
# We can view it as a rank-4 tensor of size 2 by 2 by 2 by 2
# Two of the legs of this tensor represent inputs and 
# the other two legs represent outputs
CNOT = jet.Tensor(["7", "4", "8", "10"], [2, 2, 2, 2])
CNOT.set_value((0, 0, 0, 0), 1) # |00> -> |00>
CNOT.set_value((0, 1, 0, 1), 1) # |01> -> |01>
CNOT.set_value((1, 0, 1, 1), 1) # |10> -> |11>
CNOT.set_value((1, 1, 1, 0), 1) # |11> -> |10>

CNOT_p = jet.Tensor(["1", "3", "2", "4"], [2, 2, 2, 2])
CNOT_p.set_value((0, 0, 0, 0), 1) # |00> -> |00>
CNOT_p.set_value((0, 1, 0, 1), 1) # |01> -> |01>
CNOT_p.set_value((1, 0, 1, 1), 1) # |10> -> |11>
CNOT_p.set_value((1, 1, 1, 0), 1) # |11> -> |10>

# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
H = jet.Tensor(["6", "7"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])
H_p = jet.Tensor(["2", "5"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

# Defining Pauli-Z Gate
pauli_z_data = [1, 0, 0, -1]
size = [2,2]
indices = ["5","6"]
Z = jet.Tensor(indices, size, pauli_z_data)

tn = jet.TensorNetwork();
tn.add_tensor(T_p)
tn.add_tensor(CNOT_p)
tn.add_tensor(H_p)
tn.add_tensor(Z)
tn.add_tensor(H)
tn.add_tensor(CNOT)
tn.add_tensor(T)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result)


[[0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]]


Given the construction of the above tensor network, we should get the following result: $\frac{1}{\sqrt{2}}(X + Y)$